In [ ]:
import re
import inflect

from datasets import load_dataset
from torch.utils.data import DataLoader

import polyglot
from polyglot.text import Text, Word

In [10]:
!polyglot download embeddings2.en
!polyglot download ner2.en

[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /Users/otakumesi/polyglot_data...
[polyglot_data]   Package embeddings2.en is already up-to-date!
[polyglot_data] Downloading package ner2.en to
[polyglot_data]     /Users/otakumesi/polyglot_data...


In [13]:
from allennlp.predictors import Predictor
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-model-2020.02.10.tar.gz")

[nltk_data] Downloading package punkt to /Users/otakumesi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/otakumesi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



/Users/otakumesi/Library/Caches/pypoetry/virtualenvs/debias-bert-MV5lOcox-py3.8/lib/python3.8/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:56: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  warnings.warn(



/Users/otakumesi/Library/Caches/pypoetry/virtualenvs/debias-bert-MV5lOcox-py3.8/lib/python3.8/site-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
Spacy models 'en_core_web_sm' not found.  Downloading and installing.


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
datasets = load_dataset('experiments/social_bias_frames.py')

Using custom data configuration default
Reusing dataset social_bias_frames (/Users/otakumesi/.cache/huggingface/datasets/social_bias_frames/default/0.0.0/1f541c4093d7d574ebc9638eeef12a0e247e339d90402c294272a414e259996b)


In [23]:
with open('data/masking_words.txt') as f:
    masking_words = [line.strip() for line in f.readlines()]

In [6]:
loader = DataLoader(datasets["train"])

In [ ]:
sentences = []
for i, data in enumerate(loader):
    text = data["post"][0]
    results = predictor.predict(sentence=text)
    words = [results["words"][i] for i, tag in enumerate(results["tags"]) if tag in ["U-PER", "U-LOC", "U-MISC"]]
    sentence = text
    for word in words:
        sentence = re.sub(word, "[MASK]", sentence, flags=re.IGNORECASE)
        sentence = re.sub(f"({word}|{word}'s)", "[MASK]", sentence, flags=re.IGNORECASE)
    sentences.append(sentence)

In [ ]:
for sent in sentences:
    print(sent)